
# DisulfideLoader.ipynb
Demonstrating the ways to access Disulfide Bonds using the proteusPy
package.

Author: Eric G. Suchanek, PhD.

Last Modification: 2025-02-15 15:00:50

In [1]:
import proteusPy as pp
from proteusPy.DisulfideBase import DisulfideList

pp.configure_master_logger("loadertest.log")
_logger = pp.create_logger("__name__", "WARNING")

Create an unfiltered ``DisulfideLoader`` with no distance constraints:

In [2]:
pdb = pp.DisulfideLoader(verbose=True, subset=False, cutoff=-1, sg_cutoff=-1)

proteusPy: INFO 2025-02-15 16:10:33,203 - proteusPy.DisulfideLoader.__post_init__ - Reading disulfides from: /Users/egs/miniforge3/envs/ppydev/lib/python3.12/site-packages/proteusPy/data/PDB_all_ss.pkl... 
proteusPy: INFO 2025-02-15 16:10:38,966 - proteusPy.DisulfideLoader.__post_init__ - Filtering with Cα cutoff -1.00: old: 175277, new: 175277
proteusPy: INFO 2025-02-15 16:10:38,968 - proteusPy.DisulfideLoader.__post_init__ - Filtering Sγ: cutoff -1.00: old: 175277, new: 175277
proteusPy: INFO 2025-02-15 16:11:00,477 - proteusPy.DisulfideClassManager.__init__ - Loading binary consensus structure list from SS_consensus_class_32.pkl
proteusPy: INFO 2025-02-15 16:11:00,480 - proteusPy.DisulfideClassManager.__init__ - Loading octant consensus structure list from SS_consensus_class_oct.pkl
proteusPy: INFO 2025-02-15 16:11:00,500 - proteusPy.DisulfideClassManager.build_classes - Creating binary SS classes...
proteusPy: INFO 2025-02-15 16:11:02,845 - proteusPy.DisulfideClassManager.build_cla

    =========== RCSB Disulfide Database Summary ============
       =========== Built: 2025-02-15 16:10:33 ===========
PDB IDs present:                 36968
Disulfides loaded:               175277
Average structure resolution:    2.19 Å
Lowest Energy Disulfide:         2q7q_75D_140D
Highest Energy Disulfide:        6vxk_801B_806B
Cα distance cutoff:              -1.00 Å
Sγ distance cutoff:              -1.00 Å
               ===== proteusPy: 0.99.2.dev1 =====


Access to the database is through the ``DisulfideLoader`` object. We can retrieve disulfides by PDB ID, integer indexing or slicing, disulfide name and by disulfide class. These will be illustrated below.

- Access by PDB ID:

In [3]:
pdb["6dmb"]

DisulfideList([<Disulfide 6dmb_203A_226A, Source: 6dmb, Resolution: 3.0 Å>,
               <Disulfide 6dmb_234A_327A, Source: 6dmb, Resolution: 3.0 Å>,
               <Disulfide 6dmb_296A_304A, Source: 6dmb, Resolution: 3.0 Å>])

- Access by numerical index:

In [ ]:
pdb[0]

- Access by slice:

In [ ]:
pdb[:5]

- Access by Disulfide Name:

In [ ]:
pdb["6dmb_203A_226A"]

- Access by Class Identifier, (specifiy the base explicity with a 'o' or 'b' suffix). If no suffix is included than base=8 is assumed, (to list the classes and their number of elements use the ``DisulfideLoader.print_classes()`` method with appropriate base:

In [ ]:
pdb["11212o"]

- Access by Class Identify without a suffix. Octant class is assumed

In [ ]:
pdb["11212"]

It's easy to display statistical information about the returned lists as follows:

- Show bond length and bond angle deviations for the first 1000 Disulfides:

In [ ]:
subset = pdb[:1000]
_ = subset.plot_deviation_histograms()

- Show the Cα-Cα and Sγ-Sγ distances for the first 1000 Disulfides:

In [ ]:
subset.plot_distances(distance_type="ca", cutoff=-1, theme="auto", log=True)

- Show the Ca-Ca distances for the entire database:

In [ ]:
pdb.plot_distances(theme="auto", log=True, distance_type="ca")

As you can see, the unfiltered database has a number of disulfides that exceed the maximum possible distance, (~8 A).

We can also display the torsion statistics in several ways:

- Statistics for a slice of disulfides:

In [ ]:
pdb[:10].display_torsion_statistics()

- Statistics for a class:
  - To list the binary classIDs use:

In [ ]:
pdb.get_class_df(base=2)

Since there are only 32 binary classes the above shows the overall distribution of Disulfides across ALL binary classes. The 8-fold (*octant*) class dataframe is *much* larger, (9697 members) and can be shown with:

```python
    pdb.get_class_df(base=8)
```


  Let's look at one of the binary classes (note: creating long lists of Disulfides takes time. The next cell takes over 12 seconds on my M3 Max MacBook Pro):

In [ ]:
pdb["02202b"].display_torsion_statistics(save=False, theme="auto")

The above shows quite large deviations for the dihedral angles. This suggests that the class is very broad in structural diversity. This is to be expected with a coarse structural filter, and was the driving reason to develop the *octant* dihedral angle quantization method.

In [ ]:
pdb["11212"].display_torsion_statistics(save=False, theme="auto")

- Statistics for a specific pdbID:

In [ ]:
pdb["6dmb"].display_torsion_statistics(theme="auto")

Finally, we can readily display either individual Disulfides or lists of them as follows:

In [ ]:
best_ss = pdb["2q7q_75D_140D"]
worst_ss = pdb["6vxk_801B_806B"]
duo = DisulfideList([best_ss, worst_ss], "bestworst")

In [ ]:
best_ss.display(style="sb")

We can display the list as multiple panels:

In [ ]:
duo.display(style="sb", light="dark")

Or we can display them overlaid onto a common coordinate frame:

In [ ]:
duo.display_overlay()

## Class Plotting

In [4]:
pdb.plot_classes_vs_cutoff(0.04, steps=50, base=8, theme="auto", verbose=False)

In [5]:
# pdb.plot_binary_to_eightclass_incidence(verbose=True)

In [6]:
pdb.plot_count_vs_class_df(base=8, theme="auto", verbose=True)

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['source', 'ss_id', 'proximal', 'distal', 'chi1', 'chi2', 'chi3', 'chi4', 'chi5', 'energy', 'ca_distance', 'cb_distance', 'sg_distance', 'phi_prox', 'psi_prox', 'phi_dist', 'psi_dist', 'torsion_length', 'rho', 'binary_class_string', 'octant_class_string'] but received: class_id